# Feature extraction pipeline: Shu *et al*, 2020

This notebook attempts to replicate the feature extraction pipeline of the following paper:

<div class="alert alert-block alert-success">
Shu, Zhen‐Yu, et al. <a href="https://onlinelibrary.wiley.com/doi/full/10.1002/mrm.28522?casa_token=Ab53WvMlODcAAAAA%3AXcgDLmq8egqW7uwd2g3jY9jIljhLu3VhIbvMWgbcfoWOxjO_9H7Arf91t2FBZDZ8E94Je4Wmrn0ZmkeZ">Predicting the progression of Parkinson's disease using conventional MRI and machine learning: An application of radiomic biomarkers in whole‐brain white matter.</a> Magnetic Resonance in Medicine 85.3 (2021): 1611-1624.</div>

### Usage

Include in the `directory` variable the name of the folder where your PPMI `.nii` data is available.

In [1]:
import os, glob

directory = "" # name of directory of PPMI images

assert os.path.exists(directory), "The directory provided does not exist."
assert len(glob.glob(f"{directory}/*.nii")) > 0, "The directory does not contain any Nifti files"

### Extract white matter masks using SPM12

In [2]:
import pkgutil, hashlib
from boutiques.descriptor2func import function

def create_file_name(content, fileType, length=7):
    """
    Creates a unique file name given a set of files in the directory

    Parameters
    ----------
    content: str
        The name of files in the directoryu

    fileType: str
        'batch' or 'job'

    length: int
        Length of the hash returned
    """

    # Import hash library
    sha256_hash = hashlib.sha256()

    # Update the hash object with the bytes of the input string
    sha256_hash.update(content.encode('utf-8'))

    # Get the hexadecimal representation of the hash
    full_hash = sha256_hash.hexdigest()

    # Truncate the hash to the desired length
    truncated_hash = "l" + full_hash[:length] + f"_{fileType}.m"

    return truncated_hash

def execute():
    """
    Creates both the job & batch files for SPM12
    """

    # Get template job file
    job_file = pkgutil.get_data(
        "livingpark_utils", os.path.join("templates", "segmentation_job.m")
    ).decode("utf-8")

    # Replace key by images
    nifti_images = glob.glob(f"{directory}/PPMI*.nii")
    images = ',\n'.join([f"'{os.path.abspath(image)},1'" for image in nifti_images])
    file_content = job_file.replace("[IMAGES]", images)

    # Write job file
    jobFileName = create_file_name(images, "job")
    with open(jobFileName, "w") as f:
        f.write(file_content)

    # Get template batch file
    batch_file_content = pkgutil.get_data(
        "livingpark_utils", os.path.join("templates", "call_batch.m")
    ).decode("utf-8").replace("[BATCH]", f"addpath('{os.getcwd()}')\n{jobFileName[:-2]}")

    # Write batch file
    batchFileName = create_file_name(images, "batch")
    with open(batchFileName, "w") as f:
        f.write(batch_file_content)

    return batchFileName


##### Execute SPM12 pipeline

In [3]:
# Get SPM scripts
batchFileName = execute()

# Execute SPM function
spm_batch = function("zenodo.6881412")
spm_batch(
    "launch",
    "-s",
    "-u",
    spm_batch_file=os.path.abspath(batchFileName),
    log_file_name=f"{os.path.abspath(batchFileName)}_log.log",
)

### Extract radiomic features

In [4]:
import nibabel as nib
import numpy as np
import SimpleITK as sitk
from tqdm import tqdm
import pandas as pd
import sys, json
sys.path.append('./code/scripts')
from RadiomicsHelper import RadiomicsHelper

def build_radiomics_df(radiomicsResults, save=True):
    '''
    Converts the results of the radiomic features to a Pandas dataframe

    Parameters
    ----------
    radiomicsResults: JSON
        Results of the radiomic features from run_radiomics() function

    save: bool
        Optional: saves the results to a CSV file
    '''

    def default(obj):
        if type(obj).__module__ == np.__name__:
            if isinstance(obj, np.ndarray):
                return obj.tolist()
            else:
                return obj.item()
        raise TypeError('Unknown type:', type(obj))

    subjectIds = []
    frames = []
    for sub in json.loads(json.dumps(radiomicsResults, default=default)).keys():
        subjectIds.append(sub)
        frames.append(pd.DataFrame.from_dict(json.loads(json.dumps(radiomicsResults, default=default))[sub], orient='index').T)

    radiomics_df = pd.concat(frames, keys=subjectIds)
    radiomics_df = radiomics_df.loc[:, ~radiomics_df.columns.str.startswith('diagnostics')].reset_index()


    radiomics_df = radiomics_df.rename(columns={"level_0":"PATNO"})
    radiomics_df = radiomics_df.drop(['level_1'], axis=1)

    if save:
        radiomics_df.to_csv("radiomics_results.csv")
    
    return radiomics_df


def run_radiomics():
    '''
    Computes all angled radiomic features at offset 1,4,7 and angles 0,45,90,135
    '''
    radiomicsResults = {}
    for imageFile in tqdm(glob.glob(f"{directory}/PPMI_*.nii")):
        patno = imageFile.split("/")[-1].split("_")[1]

        # Get every mask per subject (c0 to c5)
        masks = []
        affines = []
        for segmentation in sorted(glob.glob(f"{directory}/c*PPMI_{patno}*.nii")):
            mask = nib.load(segmentation)
            affines.append(mask.affine)
            masks.append(mask.get_fdata())

        # Combine all masks in one array
        maskArray = np.array(masks)

        if len(maskArray)==0:
            continue

        # Get all indices (0 to 5)
        indices = np.argmax(maskArray, axis=0)

        # Replace classes that are not WM to 0
        indices[indices!=1] = 0.0
        indices = indices.astype(np.float64)

        # Save as NifTi
        new_mask = nib.Nifti1Image(indices, affine=affines[1]) 
        mask_path = os.path.join(directory, f"wm_{patno}.nii")
        nib.save(new_mask, mask_path)

        # Compute radiomic features and save to dict
        try:
            print(f"Working on PATNO {patno}")
            helper = RadiomicsHelper(
                sitk.ReadImage(imageFile), 
                sitk.ReadImage(mask_path),
                binCount=32,
                distance=[1]
            )
            radiomicsResults[patno] = helper.getAllFeatures()
            
            helper = RadiomicsHelper(
                sitk.ReadImage(imageFile), 
                sitk.ReadImage(mask_path),
                binCount=32,
                distance=[4]
            )
            radiomicsResults[patno].update(helper.getAllFeatures())
            
            helper = RadiomicsHelper(
                sitk.ReadImage(imageFile), 
                sitk.ReadImage(mask_path),
                binCount=32,
                distance=[7]
            )
            radiomicsResults[patno].update(helper.getAllFeatures())

            os.remove(mask_path) # remove wm.nii after done using

        except Exception as e:
            print(e)

    return radiomicsResults

In [5]:
radiomicsResults = run_radiomics()
build_radiomics_df(radiomicsResults)